#Sammanfattning
Detta projekt genomfördes i samband med en kurs på Umeå Universitet och ämnar att skapa en text-generator som med hjälp av data från August Strindbergs "Inferno" ska generera text i Strindberg-stil.
Detta är mitt allra först RNN-projekt, med anledning av detta kommer koden till stor del följa den redan existerande exempel-koden som ges av Keras. Jag har själv försökt utveckla denna där jag kunnat och på ett övergripande sätt förklara det som pågår i koden. RNN står för Recurrent Neural Network och är en typ av nätverk som passar sig bra för just sekventiell data vilket är vad vi kommer hålla på med i den här koden. Utöver text så passar sig RNN bra för annan typ av sekventiell data, såsom t.ex börsdata och aktiekurser (olika typer av tidserier)m.m. I detta program kommer vi att använda en variant av RNN som kallas  LSTM (Long Short-term Memory). LSTM syftar till att lösa "the vanishing gradient problem" som är ett problem som ofta uppstår i RNNs där gradients blir försvinnande små vilket leder till mindre och mindre ändringar på viktningen av noderna i nätverket som i förlängningen leder till en inlärning som minskar och minskar. Denna LSTM-teknik gör sig väldigt bra inom NLP. LSTM har en förmåga att komma ihåg längre sekvenser jämfört med andra typer av RNN. Så i en längre mening där viktig information finns i början av meningen och där nätverket ska förutsäga slutet av meningen presterar LSTM generellt bättre.



# Importerar bibliotek
Importerar de olika biblioteken som skall komma att utnyttjas senare i programmet.

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
#from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
#import io
import requests
import re

#Hämtar och förbereder datan
Datan hämtas och bearbetas i olika numrerade steg som beskrivs nedan:

1. Hämtar datan via länk.

2. Ändrar samtliga bokstäver till gemener.

3. Skriver ut den delen av texten som senare kommer att tas bort.

4. Ignorerar inledningen av texten pga. irrelevans för det vi vill skapa.

5. Skriver ut längden av den kvarvarande texten (med tecken som enhet).

6. Hämtar och sparar samtliga unika tecken i en lista vid namn chars.

7. Skriver ut antalet unika tecken med hjälp av längden på listan chars.

8. För att det neurala nätverket ska kunna behandla vår data behöver vi först omvandla våra olika tecken till siffror. Därför gör vi två dictionaries:
ett där varje tecken får en motsvarande siffra som kommer att representera tecknet genom det neurala nätverket. I detta första dictionary kommer tecknet att vara nyckeln och siffran blir därmed värdet. I det andra dictionariet gör vi precis tvärtom för att att lättare få åtkomst till datan efter den har behandlats av det neurala nätverket.

9. Ställer in längden på sekvenserna(som blir nätverkets features) samt längden på steget mellan sekvenserna(vilket blir längden på nätverkets labels).

10. Skapar en array för alla features och en array för alla labels. Delar sedan upp texten i features och labels med hjälp av en for-loop.

11. Datan omformas till ett binärt vektor-format för att kunna hanteras av nätverket(One-hot encoding). Detta sker bl.a. genom användning av numpys zeros-funktion som returnerar en vektor med en form och datatyp som anges i argumenten.

In [ ]:
#1.
r = requests.get("http://www.gutenberg.org/cache/epub/29935/pg29935.txt")

#2.
text = r.text.lower()

#3.
print(text[0:2575]);

#4.
text=text[2575:]

#5.
print('Length of text:', len(text), "characters")

#6.
chars = sorted(list(set(text)))

#7.
print('total unique chars:', len(chars), " characters")

#8.
char_to_num = dict((c, i) for i, c in enumerate(chars))
num_to_char = dict((i, c) for i, c in enumerate(chars))

#9.
seq_length = 40
step = 3

#10.
features = []
labels = []
for i in range(0, len(text) - seq_length, step):
    features.append(text[i: i + seq_length])
    labels.append(text[i + seq_length])
print('number of sequences:', len(features))
print("length of one feature", len(features[0]))
print("length of one label", len(labels[0]))

#11.
x = np.zeros((len(features), seq_length, len(chars)), dtype=np.bool)
y = np.zeros((len(features), len(chars)), dtype=np.bool)
for i, sentence in enumerate(features):
    for t, char in enumerate(sentence):
        x[i, t, char_to_num[char]] = 1
    y[i, char_to_num[labels[i]]] = 1


﻿the project gutenberg ebook of inferno, by august strindberg

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.org


title: inferno

author: august strindberg

release date: september 8, 2009 [ebook #29935]

language: swedish


*** start of this project gutenberg ebook inferno ***




produced by jens sadowski, and projekt runeberg for
providing the scanned facsimiles.





transcribers note:

this e-text was produced from project runeberg's
digital facsimile edition of
  samlade verk #28: inferno och legender
printed in 1914 and available at http://runeberg.org/strindbg/inferno/

strindberg wrote this book originally in french. the swedish translation
was done by eugène fahlstedt.

this text has been edited so that this document only contains the book
inferno.
the table of content has been m

# Skapar modellen

Definerar modellen som sekventiell, Ger den två lager, Ett LSTM-lager för att hantera sekvenserna och ett Dense-lager för output(en output-neuron för varje unikt tecken). Dessa outputs omvandlas sedan till sannolikhetsvärden via softmax-funktionen. Båda dessa lager följer standard modellen som angivits i Google-guiderna. Utöver dessa lager så adderas en optimiserare(RMSprop) för att snabbare hitta det globala minimum där cost-funktionen har sitt lägsta möjliga värde. Även denna anges med någorlunda standardiserade parametrar där lr(learning rate) sattes till 0,01 (ju högre learning rate desto mer drastiska blir ändringarna på weights och biases). rho sätts till 0,9(också detta enligt Keras exemplet).

Till sist så anges också modellens loss-funktion som är av typen categorical crossentropy vilket är det som passar bäst för denna typ utav klassifikationsproblem. Denna modell är nu redo att tränas på att givet en sekvens av bokstäver (representerade som siffror) kunna förutsäga kommande bokstäver(också dem representerade som siffror). På detta sätt kommer modellen senare givet en liten input(randomiserad) att på egen hand generera text!
Det kommer att gå genom att modellen tränas på den träningsdata som tidigare matats in. Där features är informationen som modellen får för att kunna gör en kvalificerad gissning och labels är svaret som sedemera förväntas. Skillnaden mellan modellens gissning och det förväntade svaret är den data som modellen sedan kommer använda för att sedan förbättra sig själv(via backpropogation som kommer att förändra weights and biases), detta sker mella varje träningsepok. Backprogationen som nätverket använder sig av kallas för BPTT, vilket står för Backpropagation Through Time.

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(lr=0.01, rho=0.9)
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

#Hittar högsta sannolikheten

Denna funktion tar den ursprungliga sannolikhetsfördelning och viktar om den för att senare returnera ett teckenindex. Detta sker genom att funktionen tar emot den tidigare sannolikhetsfördelningen av de olika tecknen samt ett temperatur-värde. För temperaturer så är det så att ett högt värde leder till att skillnaderna mellan de olika sannolikheterna inte blir så stora medan ett lägre värde gör att nätverket blir mer "säker på sin sak" och att skillnaderna på sannolikheterna växer. Dvs. att temperaturen är det som styr omviktandet av vår sannolikhetsfördelning. Funktionen säkerställer sedan att när alla dessa sannolikheter adderas så blir dem tillsammans 1.0. Därefter så tar funktionen och returnerar det högsta värdet för alla uppskattade sannolikheter.




In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#Genererar texten

Denna funktion aktiveras efter varje epok och tar alltså emot den senaste epoken som argument och genererar sedemera texten. Först skriver funktionen ut vilken epok som den för tillfället använder sig av. Sen skapar funktionen ett seed som behövs för att senare trigga igång genereringen. Detta seed skapas genom en randomiserad siffra som tas fram av den inbyggda randint-funktionen. Denna funktion ges två argument: lägsta möjliga siffra och högsta möjliga siffra. Den randomiserade siffran ska senare användas för att ge nätverket en pseudorandomiserad sekvens(seed). 
Vi tar och beskriver funktionen stegvis:

1. Skriver ut vilken epok som nätverk ligger på.

2. Tar fram randomiserad siffra.

3. Anger en loop med varierande temperaturer. Samt skriver ut aktuell temperatur för varje iteration.

4. Skapar tom sträng för genererad text. Hämtar en sekvens(seed) från texten utifrån den genererade siffran. Denne sekvens läggs sedan till i den strängen för genererad text. Skriver ut aktuellt seed. 

5. Skapar en for-loop med 400 iterationer. Där varje iteration genererar en ny bokstav.

6. Vi konverterar tecken till index.

7. Genererar sannolikhetsfördelningen.

8. Skickar in sannolikhetsfördelningen samt aktuell temperatur till sample-funktionen som i sin tur returnerar indexet för det tecken med högst sannolikhet.

9. Konverterar det returnerade indexet till ett tecken med hjälp av det tidigare skapade dictionariet.

10. Tar bort ett tecken från vår sekvens(seed) och lägger till det nya tecknet.

11. Skriver ut det nya tecknet.

In [ ]:
def on_epoch_end(epoch, _):
    #1.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    #2.
    start_index = random.randint(0, len(text) - seq_length - 1)

    #3.
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        #4.
        generated_text = ''
        sentence = text[start_index: start_index + seq_length]
        generated_text += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated_text)
        #5.
        for i in range(400):

            #6.
            x_pred = np.zeros((1, seq_length, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_to_num[char]] = 1.

            #7.
            preds = model.predict(x_pred, verbose=0)[0]
            
            #8. 
            next_index = sample(preds, diversity)

            #9.
            next_char = num_to_char[next_index]

            #10.
            sentence = sentence[1:] + next_char

            11.
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

#Träningen

Här sätter vi igång träningen via model.fit som i sin tur anropar print_callback som med hjälp av ett LambadCallback talar om för programmet att anropa funktionen on_epoch_end vid varje epok-slut. Utöver detta så anger vi också batch_size och antalet epoker.

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
101179/101179 [==============================] - 72s 714us/step - loss: 2.2601

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "g form
hopknäpptes till en bönfallande "
mig mig min som min fråg mig min som mig mig mig mig mig mig med mig som han min tocker mig mig som mig mig en mig för den för mig mig mig min tocker mig mig mig min för min till min förstra sin tockan som mig för det det min var mig för den som min för strada för det mig min förstrade och som min för mig för det min till min till min som min för mig min to                                         
----- diversity: 0.5
----- Generating with seed: "g form
hopknäpptes till en bönfallande "




order för starer av till med kor mig för det nänge sin som för en sinna det och lärda som frår har sina som med mig till min
----- diversity: 1.0
----- Generating with seed: "g form
hopknäpptes till en bönfallande "



bödjag sesför
----- diversity: 1.2
----- Generating with seed: "g 

KeyboardInterrupt: ignored